이 챕터에서는 Missing data를 다루는 총 3가지 방법에 대해 공부한다.

### **1. A simple option : Drop cloumns with missing values**
가장 간단하게 결측값이 있는 열을 아예 없애 버리는 방법이다.

### **2.A better option : Imputation**
결측값을 임의의 다른 값으로 채우는 것이다.(예를 들어 평균)  
정확하지는 않겠지만 전체 열을 삭제하는 것보다는 더 정확한 모델을 만들 수 있을 것이다.

### **3.An extension to imputation**
2번의 방법에서 결측값인지 아닌지에 대한 열을 하나 추가하여 조금 더 개선시킬 수 있다.

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("../datafile/melb_data.csv")

y = data.Price

melb_predictors = data.drop(['Price'],axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)

* drop() : 값을 없애는 것
* axis : **axis = 1**은 열방향 / **axis=0**은 행방향
* select_dtypes : dtype중에 선택하는 것, 매개변수로는 include=,exclude=가 있고 말그대로
* train_test_split : train_size와 test_size는 각각 학습과 테스트할 데이터의 비율

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10,random_state=0)
    model.fit(X_train,y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

* randomforestregressor : n_estimators는 생성할 트리의 개수이다.

### **1번 방법**

In [10]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print(score_dataset(reduced_X_train,reduced_X_valid,y_train,y_valid))

183550.22137772635


* isnull().sum() : null 개수
* isnull().all() : 데이터가 모두 null인지
* isnull().any() : null이 하나라도 있는지

### **2번 방법**

In [11]:
from sklearn.impute import SimpleImputer


my_imputer = SimpleImputer()
#imputation
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

#inputation하게되면 열의 이름이 사라져서 다시 넣어줘야함
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print(score_dataset(imputed_X_train,imputed_X_valid,y_train,y_valid))


178166.46269899711


### **3번 방법**

In [12]:
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

for col in cols_with_missing:
    X_train_plus[col+'_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col+'_was_missing'] = X_valid_plus[col].isnull()

my_imputer = SimpleImputer()

imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns 

print(score_dataset(imputed_X_train_plus,imputed_X_valid_plus,y_train,y_valid))

178927.503183954
